In [43]:
# Importing necessary libraries

import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
import pickle
from sklearn.preprocessing import StandardScaler

# Extract data
data = pd.read_csv('../../ETL/Load/stockData.csv', header=(0), sep=';')

# Replace inf values by very large number
data.replace(np.inf, 1e20, inplace=True)

# Separate train data
# Train: first 8 years
data['DATE'] = pd.to_datetime(data['DATE'])
data_train = data[data['DATE'].dt.year <= 2021]

# Change data to numpy
data_train = data_train.to_numpy()

# Get number of rows and columns
ncol = data.shape[1]

# Get arrays for features and class
y_train = data_train[:,-1]
X_train = data_train[:,2:ncol-1]

# Scale the data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# Convert y to integer
y_train = y_train.astype(int)

# Set the number of rules
number_of_rules = 2

# Subtractive clustering with radius = 0.2
def subtractive_clustering(X, number_of_rules, radius=0.2):
    # Compute pairwise distances between points
    d = pdist(X, 'euclidean')
    d_squareform = squareform(d) # Distance matrix between points
    # Calculate potential for each point
    potential = np.sum(np.exp(-d_squareform**2 / (radius/2**2)), axis=1)
    # Sort by potential and select highest point
    idx = np.argsort(potential)[::-1][0]
    # Start rule center with highest potential point as a cluster center
    rule_centers = X[idx]
    # If number of rules is larger than 1, continue algorithym for the following cluster centers
    if number_of_rules > 1:
        for i in range(number_of_rules-1):
            potential_star = potential[idx]
            X_star = X[idx]
            distances = np.sqrt(np.sum((X - X_star) ** 2, axis=1))
            potential = potential - potential_star*np.exp(-distances**2 / (radius/2**2))
            X_star = X[idx]
            idx = np.argsort(potential)[::-1][0]
            rule_centers = np.vstack((rule_centers, X[idx]))
    return rule_centers


# Apply clustering to get rule centers
rule_centers = subtractive_clustering(X_train, number_of_rules, radius=0.2)
print("Rule Centers:", rule_centers)
print(type(rule_centers))

Rule Centers: [[ 0.03938546 -0.05611092  0.06268504 -0.08145881 -0.05633687 -0.05524024
  -0.44825956 -0.25503524 -0.13913047 -0.11405156 -0.03521143]
 [ 0.03938067 -0.05340169  0.10114783  0.19014632 -0.05574183 -0.05320303
  -0.4138486  -0.22370062 -0.13578946 -0.11405156 -0.02256604]]
<class 'numpy.ndarray'>


In [2]:
print(mf)

[[['gaussmf', {'mean': 0.03938546467882528, 'sigma': 1.0}], ['gaussmf', {'mean': 0.0393634760079587, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.05611092434842267, 'sigma': 1.0}], ['gaussmf', {'mean': -0.05611092434842267, 'sigma': 1.0}]]]


In [3]:
mfs = []
for i in range(X_train.shape[1]):
    mf = []
    for j in range(number_of_rules):
        mf.append(['gaussmf', {'mean': rule_centers[j][i], 'sigma': 1.0}])
    mfs.append(mf)

In [4]:
print(mfs)

[[['gaussmf', {'mean': 0.03938546467882528, 'sigma': 1.0}], ['gaussmf', {'mean': 0.0393634760079587, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.05611092434842267, 'sigma': 1.0}], ['gaussmf', {'mean': -0.05611092434842267, 'sigma': 1.0}]], [['gaussmf', {'mean': 0.06268503658898357, 'sigma': 1.0}], ['gaussmf', {'mean': 0.06268503658898357, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.08145880873610509, 'sigma': 1.0}], ['gaussmf', {'mean': -0.08145880873610509, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.05633687074392936, 'sigma': 1.0}], ['gaussmf', {'mean': -0.056330170080381235, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.0552402425359551, 'sigma': 1.0}], ['gaussmf', {'mean': -0.055278934565831245, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.448259563127543, 'sigma': 1.0}], ['gaussmf', {'mean': -0.448259563127543, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.2550352432599483, 'sigma': 1.0}], ['gaussmf', {'mean': -0.2550352432599483, 'sigma': 1.0}]], [['gaussmf', {'mean': -0.13913047128304148, 'sigm

In [5]:
number_of_rules

2

In [6]:
range(number_of_rules)

range(0, 2)

In [7]:
for j in range(number_of_rules):
    print(j)

0
1


In [8]:
rule_centers

array([[ 0.03938546, -0.05611092,  0.06268504, -0.08145881, -0.05633687,
        -0.05524024, -0.44825956, -0.25503524, -0.13913047, -0.11405156,
        -0.03521143],
       [ 0.03936348, -0.05611092,  0.06268504, -0.08145881, -0.05633017,
        -0.05527893, -0.44825956, -0.25503524, -0.13913047, -0.11405156,
        -0.03521143]])